In [1]:
# @title Setup

competition = "audio understanding"  # @param
# @markdown ---

from google.colab import userdata
import json

# Get the Kaggle credentials from Colab's userdata
username = userdata.get("KAGGLE_USER")
key = userdata.get("KAGGLE_KEY")

# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

competition_name = "audio-understanding"
dir_name = "audio_understand"

!kaggle competitions download -c {competition_name}
!mkdir {dir_name}
!unzip /content/{competition_name}.zip -d {dir_name}
!rm -rf audio-understanding.zip

 89% 350M/395M [00:01<00:00, 249MB/s]
100% 395M/395M [00:01<00:00, 302MB/s]
Archive:  /content/audio-understanding.zip
  inflating: audio_understand/speechs/speechs/test/015a21ca-7e23-4b70-93e6-da8a1bb8eaab.wav  
  inflating: audio_understand/speechs/speechs/test/02164b42-47ac-4d45-aa3a-0288ede4380e.wav  
  inflating: audio_understand/speechs/speechs/test/0225a06d-7949-49a6-84a0-0f4c2d9005aa.wav  
  inflating: audio_understand/speechs/speechs/test/02270177-6507-4730-b034-404aea8f8a5c.wav  
  inflating: audio_understand/speechs/speechs/test/02c4081a-4df6-45bc-a24d-58efa0d70c1b.wav  
  inflating: audio_understand/speechs/speechs/test/03d93186-3c38-4e6e-925f-152d489ae8ef.wav  
  inflating: audio_understand/speechs/speechs/test/05d3d878-5ecf-4dd8-9663-ccfa7afaa9da.wav  
  inflating: audio_understand/speechs/speechs/test/0620b87a-52ed-46ea-b0bb-8d722b46e55a.wav  
  inflating: audio_understand/speechs/speechs/test/085f8dec-23c1-433c-a3a6-007fe451aef0.wav  
  inflating: audio_understand/speec

In [5]:
!pip install -q pythainlp
!pip install -q deepcut attacut

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 92.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 142.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 147.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import pandas as pd
import numpy as np
import re

from tqdm import tqdm
import requests
import warnings
from pythainlp.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, f1_score, fbeta_score, accuracy_score

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy.char'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv('/content/drive/MyDrive/SAI5_Onsite/Hack04_SCBx/dataset/train_merged_transcription.csv')
test_df = pd.read_csv('/content/drive/MyDrive/SAI5_Onsite/Hack04_SCBx/dataset/asr_results_6_reordered.csv')
submission_df = pd.read_csv('/content/audio_understand/test.csv')

print(f"Training data: {df.shape}")
print(f"Test data: {test_df.shape}")
print(f"Submission template: {submission_df.shape}")

Training data: (100, 10)
Test data: (300, 2)
Submission template: (300, 9)


In [ ]:
def segment_text(text):
    if not text:
        return ""

    try:
        words = attacut.tokenize(text)
        words = [word.strip() for word in words if len(word.strip()) > 1]
        return ' '.join(words)
    except:
        return text

# tokenize message
df['tokens'] = df['transcript'].apply(segment_text)
df_clean = df[df['tokens'].str.len() > 0].copy()

test_df['tokens'] = test_df['transcript'].apply(segment_text)
test_df_clean = test_df[test_df['tokens'].str.len() > 0].copy()

# check shape
print(f"Original: {df_clean['transcript'].iloc[0][:100]}...")
print(f"Segmented: {df_clean['tokens'].iloc[0][:100]}...")

Cleaning and segmenting training data...
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
16/16 ━━━━━━━

In [29]:
df

,id,first_name,last_name,กล่าวสวัสดี,แนะนำชื่อและนามสกุล,บอกประเภทใบอนุญาตและเลขที่ใบอนุญาตที่ยังไม่หมดอายุ,บอกวัตถุประสงค์ของการเข้าพบครั้งนี้,เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้,บอกระยะเวลาที่ใช้ในการเข้าพบ,transcript,text_clean,text_segmented
0,d76f8990-f74f-4f11-8bc1-ada4928725ad,วิภาดา,มงคลเกษตริน,False,False,False,True,True,True,วิพดา หรือเรียกว่าวิว ที่ปรึกษาจากธนาคารไทยพาณ...,วิพดา หรือเรียกว่าวิว ที่ปรึกษาจากธนาคารไทยพาณ...,วิพดา หรือ เรียก ว่า วิว ที่ปรึกษา จาก ธนาคารไ...
1,614f96ee-2ea3-4d6a-824d-7b19c59526d8,พิมพ์อร,ชาละนันท์,False,False,False,False,False,True,ฉันชื่อพิมออนนะคะ เรียกออนได้เลย วันนี้ขอเวลาป...,ฉันชื่อพิมออนนะคะ เรียกออนได้เลย วันนี้ขอเวลาป...,ฉัน ชื่อ พิมออน นะ คะ เรียก ออน ได้ เลย วัน นี...
2,70cb44f4-3ec4-4b79-919f-543e67fe7959,สุดารัตน์,เกตุเลฃา,False,False,True,False,False,True,ฉันสุดารัฐชื่อเล่นดาวค่ะ เป็นที่ปรึกษาทางการเง...,ฉันสุดารัฐชื่อเล่นดาวค่ะ เป็นที่ปรึกษาทางการเง...,ฉัน สุดา รัฐ ชื่อ เล่น ดาว ค่ะ เป็น ที่ปรึกษา ...
3,a478f079-1311-4e32-9038-12afbe70375c,จิตตราพร,ชมะนันทน์,True,False,False,False,False,False,สวัสดีค่ะ คุณสิริพัฒน์ ดิฉันชื่อจิตราพรค่ะ เป็...,สวัสดีค่ะ คุณสิริพัฒน์ ดิฉันชื่อจิตราพรค่ะ เป็...,สวัสดี ค่ะ คุณสิริพัฒน์ ดิฉัน ชื่อ จิตรา พร ค่...
4,e5fbd20a-58f4-4bd7-a0c3-9c0985789a67,นคนันทินี,ชพานนท์,True,False,False,False,True,True,สวัสดีค่ะ คุณสิริภา นีน่าจัดธนาคารไทยพาณิชนะคะ...,สวัสดีค่ะ คุณสิริภา นีน่าจัดธนาคารไทยพาณิชนะคะ...,สวัสดี ค่ะ คุณสิริภา นีน่า จัด ธนาคารไทยพาณิช ...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,59d727b3-cec0-4f8a-898f-47ae1b75a530,จารุนันท์,เกียรติภัชรสกุล,False,False,False,False,False,False,ดิฉันชื่อจารุนังค่ะ เรียกว่าจูนก็ได้ เป็นที่ปร...,ดิฉันชื่อจารุนังค่ะ เรียกว่าจูนก็ได้ เป็นที่ปร...,ดิฉัน ชื่อ จารุ นัง ค่ะ เรียก ว่า จูน ก็ได้ เป...
96,40915b1d-5efb-461a-8044-aa84ce9232c3,เดชา,กรัพณานนท์,True,False,True,True,True,False,สวัสดีค่ะคุณสุชาวดี ชื่อผมเดชค่ะ เป็นที่ปรึกษา...,สวัสดีค่ะคุณสุชาวดี ชื่อผมเดชค่ะ เป็นที่ปรึกษา...,สวัสดี ค่ะ คุณสุชาว ดี ชื่อ ผม เดช ค่ะ เป็น ที...
97,ebcf716e-d2e0-488a-a8ff-e5c2f6502f19,สุริวิภา,เมนะเนตร,False,False,True,False,False,True,ดิฉันสุริวิภา เมนเนศ เป็นที่ปรึกษาทางการเงินจา...,ดิฉันสุริวิภา เมนเนศ เป็นที่ปรึกษาทางการเงินจา...,ดิฉัน สุริวิภา เมนเนศ เป็น ที่ปรึกษา ทาง การ เ...
98,f0f823be-d2b5-4548-8f1d-5d66a1c29346,ชัชชนม์,อันตรนิยุกต์,True,False,True,False,False,False,สวัสดีครับคุณสุพินธิผมชื่อชัชนงหรือเรียกว่าชัด...,สวัสดีครับคุณสุพินธิผมชื่อชัชนงหรือเรียกว่าชัด...,สวัสดี ครับ คุณสุพินธิ ผม ชื่อ ชัชนง หรือ เรีย...


In [23]:
def extract_thai_names(text):
    if not text:
        return False

    patterns = [
        r'(?:ผม|ดิฉัน|กระผม)(?:ชื่อ)?\s*([ก-๙]+)\s+([ก-๙]+)',
        r'(?:ชื่อ)\s*([ก-๙]+)\s+([ก-๙]+)',
        r'([ก-๙]+)\s+([ก-๙]+)(?:\s+(?:ครับ|ค่ะ|คะ))'
    ]

    for pattern in patterns:
        matches = re.findall(pattern, text)
        if matches:
            for first, last in matches:
                if len(first) >= 2 and len(last) >= 2:
                    return True
    return False


test_ner_results = []
for text in tqdm(test_df_clean['text_clean']):
    test_ner_results.append(extract_thai_names(text))


100%|██████████| 300/300 [00:00<00:00, 3742.20it/s]


In [27]:
def create_keyword_features(texts):
    keywords = {
        'greeting': ['สวัสดี', 'หวัดดี', 'ดี', 'ครับ', 'ค่ะ'],
        'license': ['ใบอนุญาต', 'เลขที่', 'หมายเลข', 'ไม่', 'หมดอายุ', 'อนุญาต'],
        'purpose': ['วัตถุประสงค์', 'เข้า', 'พบ', 'พูดคุย', 'เพื่อ', 'มา'],
        'benefit': ['ประโยชน์', 'ได้รับ', 'ช่วย', 'ให้', 'ดี', 'ขึ้น'],
        'time': ['เวลา', 'นาที', 'ชั่วโมง', '20', '30', '60', 'ครู่']
    }

    features = []
    for text in texts:
        row = []
        for category, words in keywords.items():
            count = sum(1 for word in words if word in text)
            row.append(count)
        features.append(row)

    return np.array(features)

print("Creating basic features with segmented text...")

# Get training text (use segmented text for BoW and TF-IDF)
X_text_segmented = df_clean['text_segmented']

# 1. Bag of Words with segmented text
print("Creating Bag of Words features with segmented text...")
bow_vectorizer = CountVectorizer(
    max_features=2000,
    ngram_range=(1, 1),
    min_df=2,
    max_df=0.95,
    lowercase=True
)
X_bow = bow_vectorizer.fit_transform(X_text_segmented)

# 2. TF-IDF with segmented text
print("Creating TF-IDF features with segmented text...")
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    lowercase=True
)
X_tfidf = tfidf_vectorizer.fit_transform(X_text_segmented)

# 3. Basic keyword features
print("Creating basic keyword features...")
X_keywords = create_keyword_features(X_text_segmented)

# 4. Combine basic features
X_combined = np.hstack([
    X_bow.toarray(),
    X_tfidf.toarray(),
    X_keywords
])

print(f"Basic feature summary:")
print(f"  BoW: {X_bow.shape[1]} features")
print(f"  TF-IDF: {X_tfidf.shape[1]} features")
print(f"  Keywords: {X_keywords.shape[1]} features")
print(f"  Total Basic: {X_combined.shape[1]} features")

Creating basic features with segmented text...
Creating Bag of Words features with segmented text...
Creating TF-IDF features with segmented text...
Creating basic keyword features...
Basic feature summary:
  BoW: 286 features
  TF-IDF: 719 features
  Keywords: 5 features
  Total Basic: 1010 features


In [ ]:
def create_enhanced_keyword_features(texts):
    """Enhanced keyword features with specific patterns for problematic columns"""

    enhanced_keywords = {
        # Purpose-related features (วัตถุประสงค์)
        'purpose_explicit': ['วัตถุประสงค์', 'จุดประสงค์', 'เป้าหมาย', 'ความต้องการ'],
        'purpose_action': ['เข้าพบ', 'มาพบ', 'มาหา', 'ติดต่อ', 'สอบถาม', 'ปรึกษา'],
        'purpose_business': ['ธุรกิจ', 'งาน', 'โครงการ', 'แผน', 'ข้อเสนอ', 'นำเสนอ'],
        'purpose_meeting': ['ประชุม', 'พูดคุย', 'แลกเปลี่ยน', 'หารือ', 'ปรึกษา'],
        'purpose_info': ['ข้อมูล', 'รายละเอียด', 'รายงาน', 'ผลงาน', 'สถานการณ์'],

        # Benefit-related features (ประโยชน์)
        'benefit_explicit': ['ประโยชน์', 'ได้รับ', 'ได้ประโยชน์', 'คุณค่า', 'มีประโยชน์'],
        'benefit_improve': ['ดีขึ้น', 'เพิ่มขึ้น', 'พัฒนา', 'ปรับปรุง', 'ยกระดับ'],
        'benefit_help': ['ช่วย', 'ช่วยให้', 'สนับสนุน', 'ส่งเสริม', 'อำนวยความสะดวก'],
        'benefit_reduce': ['ลด', 'ลดลง', 'ประหยัด', 'ลดต้นทุน', 'ลดปัญหา'],
        'benefit_solve': ['แก้ไข', 'แก้ปัญหา', 'ปรับแต่ง', 'แก้', 'จัดการ'],
        'benefit_business': ['เติบโต', 'ขยาย', 'เพิ่มผล', 'เพิ่มรายได้', 'มีกำไร'],
        'benefit_efficiency': ['มีประสิทธิภาพ', 'เร็วขึ้น', 'สะดวก', 'ง่าย', 'คุ้มค่า'],

        # Enhanced original categories
        'greeting_enhanced': ['สวัสดี', 'หวัดดี', 'ดี', 'ครับ', 'ค่ะ', 'สบายดี', 'ยินดี'],
        'license_enhanced': ['ใบอนุญาต', 'เลขที่', 'หมายเลข', 'ไม่', 'หมดอายุ', 'อนุญาต', 'ใบ', 'บัตร'],
        'time_enhanced': ['เวลา', 'นาที', 'ชั่วโมง', '20', '30', '60', 'ครู่', 'ประมาณ', 'ใช้เวลา']
    }

    features = []
    for text in texts:
        row = []
        for category, words in enhanced_keywords.items():
            # Count exact matches
            exact_count = sum(1 for word in words if word in text)

            # Add weighted score based on category importance
            if 'purpose' in category:
                weight = 1.5  # Higher weight for purpose-related
            elif 'benefit' in category:
                weight = 1.3  # Higher weight for benefit-related
            else:
                weight = 1.0

            row.append(exact_count * weight)
        features.append(row)

    return np.array(features)

def create_pattern_features(texts):
    """Create pattern-based features for specific detection"""

    patterns = {
        # Purpose patterns
        'purpose_sentence': [
            r'วัตถุประสงค์.*?(?:คือ|ของ|การ)',
            r'(?:มา|เข้า).*?(?:เพื่อ|เพื่อที่จะ)',
            r'ต้องการ.*?(?:ที่จะ|การ)',
            r'จุดประสงค์.*?(?:ของ|คือ)'
        ],

        # Benefit patterns
        'benefit_sentence': [
            r'(?:ได้|จะได้).*?ประโยชน์',
            r'ประโยชน์.*?(?:คือ|ที่|จาก)',
            r'(?:ช่วย|ช่วยให้).*?(?:ดีขึ้น|เพิ่ม)',
            r'(?:ลด|ลดลง).*?(?:ต้นทุน|ปัญหา)',
            r'(?:แก้|แก้ไข).*?(?:ปัญหา|สถานการณ์)'
        ],

        # Context patterns
        'formal_context': [
            r'(?:ผม|ดิฉัน|กระผม).*?(?:มา|เข้า)',
            r'(?:ครับ|ค่ะ|คะ).*?(?:วัน|เวลา)',
            r'(?:ขอ|ขออนุญาต).*?(?:เข้า|พบ)'
        ]
    }

    features = []
    for text in texts:
        row = []
        for category, pattern_list in patterns.items():
            count = 0
            for pattern in pattern_list:
                matches = re.findall(pattern, text, re.IGNORECASE)
                count += len(matches)
            row.append(count)
        features.append(row)

    return np.array(features)

print("Creating enhanced column-specific features...")

# Create enhanced features for both training and test
enhanced_keywords = create_enhanced_keyword_features(df_clean['text_segmented'])
pattern_features = create_pattern_features(df_clean['text_clean'])  # Use original text for patterns

# Combine enhanced features
X_enhanced_specific = np.hstack([
    enhanced_keywords,
    pattern_features
])

print(f"Enhanced specific features: {X_enhanced_specific.shape}")

# Final combined features (Basic + Enhanced)
X_final_combined = np.hstack([
    X_combined,           # Basic features
    X_enhanced_specific   # Enhanced specific features
])

print(f"Final combined features: {X_final_combined.shape}")

Creating enhanced column-specific features...
Enhanced specific features: (337, 18)
Final combined features: (337, 2713)


In [ ]:
target_columns = [
    'กล่าวสวัสดี',
    'บอกประเภทใบอนุญาตและเลขที่ใบอนุญาตที่ยังไม่หมดอายุ',
    'บอกวัตถุประสงค์ของการเข้าพบครั้งนี้',
    'เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้',
    'บอกระยะเวลาที่ใช้ในการเข้าพบ'
]

# Convert to numpy array
y_target = df_clean[target_columns].values
print(f"Target variables: {y_target.shape}")

Target variables: (337, 5)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_final_combined, y_target,
    test_size=0.2,
    random_state=42,
    stratify=y_target[:, 0]  # Stratify by first column
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")

Training set: (269, 2713)
Validation set: (68, 2713)


In [ ]:
basic_models = {
    'RandomForest': RandomForestClassifier(
        n_estimators=100,
        max_depth=12,
        min_samples_split=10,
        min_samples_leaf=5,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),
    'ExtraTrees': ExtraTreesClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=15,
        min_samples_leaf=7,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=50,
        max_depth=6,
        learning_rate=0.1,
        min_samples_split=20,
        min_samples_leaf=10,
        subsample=0.8,
        random_state=42
    ),
    'LogisticRegression': LogisticRegression(
        C=0.1,
        penalty='l2',
        max_iter=1000,
        random_state=42
    ),
    'LightGBM': LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        num_leaves=64,
        learning_rate=0.05,
        n_estimators=200,
        max_depth=8,
        min_child_samples=20,
        min_child_weight=1e-3,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        verbose=-1,
        n_jobs=-1,
        force_row_wise=True
    )
}

# Column-specific models for problematic columns
def create_column_specific_models():
    """Create specialized models for different column types"""

    # Model for purpose detection
    purpose_model = LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        num_leaves=128,
        learning_rate=0.03,
        n_estimators=300,
        max_depth=10,
        min_child_samples=15,
        subsample=0.85,
        colsample_bytree=0.9,
        reg_alpha=0.05,
        reg_lambda=0.05,
        random_state=42,
        verbose=-1,
        n_jobs=-1
    )

    # Model for benefit detection
    benefit_model = LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        num_leaves=96,
        learning_rate=0.04,
        n_estimators=250,
        max_depth=8,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.85,
        reg_alpha=0.08,
        reg_lambda=0.08,
        random_state=42,
        verbose=-1,
        n_jobs=-1
    )

    return {
        'บอกวัตถุประสงค์ของการเข้าพบครั้งนี้': purpose_model,
        'เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้': benefit_model
    }

# Create basic ensemble models
ensemble_models = {}
for name, model in basic_models.items():
    ensemble_models[name] = MultiOutputClassifier(model)

In [ ]:
trained_basic_models = {}
for name, model in ensemble_models.items():
    print(f"Training basic {name}...")
    model.fit(X_train, y_train)
    trained_basic_models[name] = model

# Train column-specific models
print("Training column-specific models...")
specific_models = create_column_specific_models()
trained_specific_models = {}

for col_name, model in specific_models.items():
    col_idx = target_columns.index(col_name)
    print(f"Training specific model for: {col_name}")
    model.fit(X_train, y_train[:, col_idx])
    trained_specific_models[col_name] = model

Training basic RandomForest...
Training basic ExtraTrees...
Training basic GradientBoosting...
Training basic LogisticRegression...
Training basic LightGBM...
Training column-specific models...
Training specific model for: บอกวัตถุประสงค์ของการเข้าพบครั้งนี้
Training specific model for: เน้นประโยชน์ว่าลูกค้าได้ประโยชน์อะไรจากการเข้าพบครั้งนี้


In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """Evaluate model performance"""
    print(f"\n{model_name} Performance:")
    print("-" * 40)

    f2_scores = []
    for i, col in enumerate(target_columns):
        f2 = fbeta_score(y_true[:, i], y_pred[:, i], beta=2)
        acc = accuracy_score(y_true[:, i], y_pred[:, i])
        f2_scores.append(f2)
        print(f"  {col[:30]}...: F2={f2:.3f}, Acc={acc:.3f}")

    avg_f2 = np.mean(f2_scores)
    print(f"  Average F2: {avg_f2:.3f}")
    return avg_f2

print("Evaluating basic models...")

# Get predictions from basic models
basic_val_predictions = {}
basic_model_scores = {}

for name, model in trained_basic_models.items():
    pred = model.predict(X_val)
    basic_val_predictions[name] = pred
    score = evaluate_model(y_val, pred, f"Basic {name}")
    basic_model_scores[name] = score

# Create hybrid predictions (basic + specific)
print("Creating hybrid predictions...")

# Start with best basic model predictions
best_basic_model = max(basic_model_scores.items(), key=lambda x: x[1])[0]
hybrid_predictions = basic_val_predictions[best_basic_model].copy()

# Replace specific columns with specialized model predictions
for col_name, model in trained_specific_models.items():
    col_idx = target_columns.index(col_name)
    specific_pred = model.predict(X_val)
    hybrid_predictions[:, col_idx] = specific_pred

hybrid_score = evaluate_model(y_val, hybrid_predictions, "Hybrid (Basic + Specific)")

Evaluating basic models...

Basic RandomForest Performance:
----------------------------------------
  กล่าวสวัสดี...: F2=0.993, Acc=0.985
  บอกประเภทใบอนุญาตและเลขที่ใบอน...: F2=1.000, Acc=1.000
  บอกวัตถุประสงค์ของการเข้าพบครั...: F2=0.903, Acc=0.868
  เน้นประโยชน์ว่าลูกค้าได้ประโยช...: F2=0.850, Acc=0.882
  บอกระยะเวลาที่ใช้ในการเข้าพบ...: F2=0.994, Acc=0.985
  Average F2: 0.948

Basic ExtraTrees Performance:
----------------------------------------
  กล่าวสวัสดี...: F2=0.993, Acc=0.985
  บอกประเภทใบอนุญาตและเลขที่ใบอน...: F2=1.000, Acc=1.000
  บอกวัตถุประสงค์ของการเข้าพบครั...: F2=0.888, Acc=0.882
  เน้นประโยชน์ว่าลูกค้าได้ประโยช...: F2=0.828, Acc=0.882
  บอกระยะเวลาที่ใช้ในการเข้าพบ...: F2=1.000, Acc=1.000
  Average F2: 0.942

Basic GradientBoosting Performance:
----------------------------------------
  กล่าวสวัสดี...: F2=0.993, Acc=0.985
  บอกประเภทใบอนุญาตและเลขที่ใบอน...: F2=1.000, Acc=1.000
  บอกวัตถุประสงค์ของการเข้าพบครั...: F2=0.897, Acc=0.853
  เน้นประโยชน์ว่าลูกค้าได้ประ

In [ ]:
total_basic_score = sum(basic_model_scores.values())
basic_weights = {name: score/total_basic_score for name, score in basic_model_scores.items()}

# Weighted ensemble of basic models
weighted_basic = np.zeros_like(hybrid_predictions)
for name, model in trained_basic_models.items():
    pred_proba = model.predict_proba(X_val)
    for i in range(weighted_basic.shape[0]):
        for j in range(weighted_basic.shape[1]):
            prob = pred_proba[j][i][1] if len(pred_proba[j][i]) > 1 else pred_proba[j][i][0]
            weighted_basic[i, j] += basic_weights[name] * prob

weighted_basic = (weighted_basic > 0.5).astype(int)

# Final ensemble: Combine weighted basic + specific models
final_ensemble = weighted_basic.copy()

# Override with specific model predictions for problematic columns
for col_name, model in trained_specific_models.items():
    col_idx = target_columns.index(col_name)
    specific_pred = model.predict(X_val)
    final_ensemble[:, col_idx] = specific_pred

# Evaluate final ensemble
weighted_basic_score = evaluate_model(y_val, weighted_basic, "Weighted Basic Ensemble")
final_ensemble_score = evaluate_model(y_val, final_ensemble, "Final Ensemble (Weighted + Specific)")

# Select best approach
all_approaches = {
    'Hybrid': hybrid_score,
    'Weighted_Basic': weighted_basic_score,
    'Final_Ensemble': final_ensemble_score
}

best_approach, best_score = max(all_approaches.items(), key=lambda x: x[1])
print(f"\nBest approach: {best_approach} (F2: {best_score:.3f})")


Weighted Basic Ensemble Performance:
----------------------------------------
  กล่าวสวัสดี...: F2=0.767, Acc=0.397
  บอกประเภทใบอนุญาตและเลขที่ใบอน...: F2=0.778, Acc=0.412
  บอกวัตถุประสงค์ของการเข้าพบครั...: F2=0.798, Acc=0.441
  เน้นประโยชน์ว่าลูกค้าได้ประโยช...: F2=0.807, Acc=0.456
  บอกระยะเวลาที่ใช้ในการเข้าพบ...: F2=0.849, Acc=0.529
  Average F2: 0.800

Final Ensemble (Weighted + Specific) Performance:
----------------------------------------
  กล่าวสวัสดี...: F2=0.767, Acc=0.397
  บอกประเภทใบอนุญาตและเลขที่ใบอน...: F2=0.778, Acc=0.412
  บอกวัตถุประสงค์ของการเข้าพบครั...: F2=0.865, Acc=0.824
  เน้นประโยชน์ว่าลูกค้าได้ประโยช...: F2=0.909, Acc=0.926
  บอกระยะเวลาที่ใช้ในการเข้าพบ...: F2=0.849, Acc=0.529
  Average F2: 0.834

Best approach: Hybrid (F2: 0.948)


In [ ]:
X_test_bow = bow_vectorizer.transform(test_df_clean['text_segmented'])
X_test_tfidf = tfidf_vectorizer.transform(test_df_clean['text_segmented'])
X_test_keywords = create_keyword_features(test_df_clean['text_segmented'])

# Basic test features
X_test_combined = np.hstack([
    X_test_bow.toarray(),
    X_test_tfidf.toarray(),
    X_test_keywords
])

# Enhanced test features
test_enhanced_keywords = create_enhanced_keyword_features(test_df_clean['text_segmented'])
test_pattern_features = create_pattern_features(test_df_clean['text_clean'])

X_test_enhanced_specific = np.hstack([
    test_enhanced_keywords,
    test_pattern_features
])

# Final test features
X_test_final = np.hstack([
    X_test_combined,
    X_test_enhanced_specific
])


In [ ]:
if best_approach == 'Hybrid':
    # Use best basic model + specific models
    best_basic_model = max(basic_model_scores.items(), key=lambda x: x[1])[0]
    final_test_pred = trained_basic_models[best_basic_model].predict(X_test_final)

    # Override with specific models
    for col_name, model in trained_specific_models.items():
        col_idx = target_columns.index(col_name)
        specific_test_pred = model.predict(X_test_final)
        final_test_pred[:, col_idx] = specific_test_pred

elif best_approach == 'Weighted_Basic':
    # Weighted ensemble of basic models
    final_test_pred = np.zeros((len(test_df_clean), len(target_columns)))
    for name, model in trained_basic_models.items():
        pred_proba = model.predict_proba(X_test_final)
        for i in range(final_test_pred.shape[0]):
            for j in range(final_test_pred.shape[1]):
                prob = pred_proba[j][i][1] if len(pred_proba[j][i]) > 1 else pred_proba[j][i][0]
                final_test_pred[i, j] += basic_weights[name] * prob
    final_test_pred = (final_test_pred > 0.5).astype(int)

else:  # Final_Ensemble
    # Weighted basic + specific models
    final_test_pred = np.zeros((len(test_df_clean), len(target_columns)))
    for name, model in trained_basic_models.items():
        pred_proba = model.predict_proba(X_test_final)
        for i in range(final_test_pred.shape[0]):
            for j in range(final_test_pred.shape[1]):
                prob = pred_proba[j][i][1] if len(pred_proba[j][i]) > 1 else pred_proba[j][i][0]
                final_test_pred[i, j] += basic_weights[name] * prob
    final_test_pred = (final_test_pred > 0.5).astype(int)

    # Override with specific models
    for col_name, model in trained_specific_models.items():
        col_idx = target_columns.index(col_name)
        specific_test_pred = model.predict(X_test_final)
        final_test_pred[:, col_idx] = specific_test_pred

In [ ]:
pred_mapping = {}
# Changed 'file' to 'id'
for i, file_id in enumerate(test_df_clean['id']):
    pred_mapping[file_id] = {
        target_columns[j]: bool(final_test_pred[i, j]) for j in range(len(target_columns))
    }

# Add NER results
# Changed 'file' to 'id'
ner_mapping = dict(zip(test_df_clean['id'], test_ner_results))

# Fill submission template
final_submission = submission_df.copy()

# Fill classification columns
for col in target_columns:
    final_submission[col] = final_submission['id'].map(
        lambda x: pred_mapping.get(x, {}).get(col, False)
    )

# Fill NER column
if 'แนะนำชื่อและนามสกุล' in final_submission.columns:
    final_submission['แนะนำชื่อและนามสกุล'] = final_submission['id'].map(
        lambda x: ner_mapping.get(x, False)
    )

In [ ]:
output_file = 'all_for_one_3.csv'
final_submission.to_csv(output_file, index=False)

print(f"Results saved to: {output_file}")

# Summary statistics
print(f"\nPrediction summary:")
for col in target_columns + ['แนะนำชื่อและนามสกุล']:
    if col in final_submission.columns:
        count = final_submission[col].sum()
        total = len(final_submission)
        pct = (count / total) * 100
        print(f"  {col[:30]}...: {count}/{total} ({pct:.1f}%)")

print(f"\nPipeline complete!")
print(f"Best approach: {best_approach}")
print(f"Best F2 score: {best_score:.3f}")

# Performance comparison
print(f"\nPerformance Comparison:")
print(f"Original Best Basic Model: {max(basic_model_scores.values()):.3f}")
print(f"Enhanced Final Ensemble: {best_score:.3f}")
print(f"Improvement: +{(best_score - max(basic_model_scores.values())):.3f}")

In [ ]:
output_file = '/content/drive/MyDrive/Hack4_ASR/BOW_TFIDF/all_for_one_3.csv'
final_submission.to_csv(output_file, index=False)

print(f"Results saved to: {output_file}")

# Summary statistics
print(f"\nPrediction summary:")
for col in target_columns + ['แนะนำชื่อและนามสกุล']:
    if col in final_submission.columns:
        count = final_submission[col].sum()
        total = len(final_submission)
        pct = (count / total) * 100
        print(f"  {col[:30]}...: {count}/{total} ({pct:.1f}%)")

print(f"\nPipeline complete!")
print(f"Best approach: {best_approach}")
print(f"Best F2 score: {best_score:.3f}")

# Performance comparison
print(f"\nPerformance Comparison:")
print(f"Original Best Basic Model: {max(basic_model_scores.values()):.3f}")
print(f"Enhanced Final Ensemble: {best_score:.3f}")
print(f"Improvement: +{(best_score - max(basic_model_scores.values())):.3f}")